In [ ]:
%matplotlib inline
%load_ext autoreload

%autoreload 2

from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
from tqdm import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
import time
from sklearn.cluster import KMeans
from collections import defaultdict
from collections import Counter
import cvxpy
import numpy as np
from sklearn.metrics import adjusted_rand_score
from sklearn.linear_model import LogisticRegression
from scipy.optimize import linprog
from copy import deepcopy


sns.reset_defaults()
sns.set_context(context='talk',font_scale=0.7)


from hdbscan._hdbscan_tree import (condense_tree,
                            compute_stability,
                            get_clusters,
                            outlier_scores,
                                   do_labelling,
                                   get_probabilities,
                                   get_stability_scores)
from hdbscan.hdbscan_ import _hdbscan_boruvka_kdtree, _hdbscan_prims_kdtree, _hdbscan_generic
from hdbscan import HDBSCAN
from cluster_combinations import MultiscaleHDBSCAN
from sklearn.metrics import (
    completeness_score, adjusted_rand_score, adjusted_mutual_info_score, fowlkes_mallows_score, homogeneity_score)

def compare_clusterings(ground_truth, c):
    # Note that we use homoeneity because we have access to the ground truth 
    return (
        adjusted_rand_score(ground_truth, c),
        completeness_score(ground_truth, c),
        fowlkes_mallows_score(ground_truth, c),
        homogeneity_score(ground_truth,c))


In [ ]:
from data_helper import load_umap_mnist, load_umap_newsgroups
X_mnist, y_mnist = load_umap_mnist(sampling=20, n_components=10)
X_news, y_news = load_umap_newsgroups(sampling=1, n_components=10)


In [ ]:
alpha_list = [1.0, 2.0, 3.0, 4.0, 5.0]
for alpha in alpha_list:
    hdbscan_labels = HDBSCAN(
        alpha=alpha,
        min_samples=5,
        min_cluster_size=100,
        algorithm="prims_kdtree",
        metric='euclidean',
        p=None,
        gen_min_span_tree=False,
        cluster_selection_method="eom",
        allow_single_cluster=False,
        match_reference_implementation=False,
        cluster_selection_epsilon=0.0,
        leaf_size=3).fit_predict(X_mnist)
    print("hdbscan {}".format(alpha), compare_clusterings(y_mnist, hdbscan_labels))

    
mhdbscan_labels = MultiscaleHDBSCAN(
    alpha_list=alpha_list,
    min_samples=5,
    m=2,
    min_cluster_size=100,
    normalize_stabilities=False,
    solver="mip").fit_predict(X_mnist)

print("mhdbscan: ", compare_clusterings(y_mnist, mhdbscan_labels))
print(Counter(mhdbscan_labels))